<a href="https://colab.research.google.com/github/lucarinelli/conditional_text_generation/blob/main/notebooks/COCO_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import utilities

In [ ]:
!pip install --quiet transformers datasets tokenizers

In [ ]:
repo_dir = "/content/conditional_text_generation"
!rm -r {repo_dir}
!git clone https://github.com/lucarinelli/conditional_text_generation.git {repo_dir}

In [ ]:
import sys
import os

module_path = os.path.abspath(repo_dir+"/src")
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from captions_dataset import *

# Configuration

In [ ]:
%mkdir "data"
data_path="./data"

# Coco Analysis

In [ ]:
def computeAverageOnDataset(dataset, fieldExtractor):
  count = 0
  minV = None
  maxV = None
  for i in dataset:
    l = len(fieldExtractor(i)) 
    count += l
    if minV is None or l < minV: minV = l
    if maxV is None or l > maxV: maxV = l
  return count / len(dataset), minV, maxV

In [ ]:
def logControlCodeAnalysis(type, use_supercategories, use_categories):
  print("Creating dataset using {}".format(type))
  dataset_train, _, _ = load_or_setup_dataset(data_path=data_path, split="train", use_supercategories = use_supercategories, use_categories = use_categories, force_dataset_update = True)
  number_of_categories = list(map(lambda e: len(e["categories"]), dataset_train))
  average, min, max = computeAverageOnDataset(dataset_train, lambda e: e["categories"])
  percentile = 100 - len(list(filter(lambda nc: nc > average, number_of_categories))) / len(dataset_train) *100

  print("For {} the average number of control codes per caption is {}.\nIt's the {:.0f}th percentile. Minimum is {}. Maximum is {}".format(type, average, percentile, min, max))

In [ ]:
logControlCodeAnalysis("supercategories only", True, False)

In [ ]:
logControlCodeAnalysis("categories only", False, True )

In [ ]:
logControlCodeAnalysis("categories and supercategories", True, True)

In [ ]:
dataset_train, _, categories = load_or_setup_dataset(data_path, "train", True, False, True)
averageChar, minC, maxC = computeAverageOnDataset(dataset_train, lambda e: e["caption"])
print("Average length of captions is {} chars. Min {} and max {}".format(averageChar, minC, maxC))
averageWords, minW, maxW = computeAverageOnDataset(dataset_train, lambda x: x["caption"].split())
print("Average length of captions is {} words. Min {} and max {}".format(averageWords, minW, maxW))

In [ ]:
print("Database has {} entries.".format(len(dataset_train)))